In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import glob
import os
from utility import *
from UnitGaussianNormalizer import *
import torch.nn.functional as F
import torch
from CustomDataset import *
import pickle
from FNO4D import *
import json
from visulization_compare import *
plt.jet()

<Figure size 432x288 with 0 Axes>

In [2]:
NORMALIZER_DICT = {}
d_in, d_out = {}, {}
for key in ['GLOBAL', 'LGR1', 'LGR2', 'LGR3', 'LGR4']:
    with open(f"normalizer/input_normalizer_{key}_DP_val.pickle", 'rb') as f:
        input_normalizer = pickle.load(f)
    with open(f"normalizer/output_normalizer_{key}_DP_val.pickle", 'rb') as f:
        output_normalizer = pickle.load(f)
    input_normalizer.cuda()
    output_normalizer.cuda()
    d_in[key] = input_normalizer
    d_out[key] = output_normalizer
NORMALIZER_DICT['input'] = d_in
NORMALIZER_DICT['output'] = d_out

print(NORMALIZER_DICT.keys())
print(NORMALIZER_DICT['input'].keys(), NORMALIZER_DICT['output'].keys())

dict_keys(['input', 'output'])
dict_keys(['GLOBAL', 'LGR1', 'LGR2', 'LGR3', 'LGR4']) dict_keys(['GLOBAL', 'LGR1', 'LGR2', 'LGR3', 'LGR4'])


In [3]:
# Change these models to your trained weights
PATH = {}
PATH['GLOBAL'] = "pre_trained_models/FNO4D-GLOBAL-DP.pt"
PATH['LGR1'] = "pre_trained_models/FNO4D-LGR1-DP.pt"
PATH['LGR2'] = "pre_trained_models/FNO4D-LGR2-DP.pt"
PATH['LGR3'] = "pre_trained_models/FNO4D-LGR3-DP.pt"
PATH['LGR4'] = "pre_trained_models/FNO4D-LGR4-DP.pt"
device = torch.device('cuda')

MODEL_DICT = {}
for key in ['GLOBAL', 'LGR1', 'LGR2', 'LGR3', 'LGR4']:
    model = torch.load(PATH[key])
    model.to(device)
    model.eval()
    MODEL_DICT[key] = model
    
print(MODEL_DICT.keys())

dict_keys(['GLOBAL', 'LGR1', 'LGR2', 'LGR3', 'LGR4'])


In [4]:
DATA_LOADER_DICT = torch.load('DATA_LOADER_DICT.pth')
train_loader = DATA_LOADER_DICT['GLOBAL']['train']
val_loader = DATA_LOADER_DICT['GLOBAL']['val']
test_loader = DATA_LOADER_DICT['GLOBAL']['test']
n_train = len(train_loader)
n_val = len(val_loader)
n_test = len(test_loader)
print(n_train, n_val, n_test)

16 2 2


In [31]:
it = iter(train_loader)
ERR_LIST = []

for counter in range(len(train_loader)):
    with torch.no_grad():
        PRED, TRUE = {}, {}
        data = next(it)
        x, y, path = data['x'], data['y'], data['path']
        x, y = x[None,...].to(device), y[None,...]
        x[...,-1:] = NORMALIZER_DICT['input']['GLOBAL'].encode(x.to(device)[...,-1:])
        pred = NORMALIZER_DICT['output']['GLOBAL'].decode(MODEL_DICT['GLOBAL'](x)).cpu()
        PRED['GLOBAL'] = pred
        TRUE['GLOBAL'] = y
        slope, idx, well = path[0], path[1], path[2]

        meta_data = np.load(f'ECLIPSE/meta_data/{slope}_{idx}.npy', allow_pickle=True).tolist()
        WELL_LIST = meta_data[f'case_{idx}']['WELL_LIST']
        GRID_IDX_DICT = meta_data[f'case_{idx}']['GRID_IDX_DICT']

        for well in WELL_LIST:
            lgr_dict, true_dict = {}, {}

            data_LGR1 = torch.load(f'dataset/dP_LGR1/{slope}_{idx}_LGR1_{well}_DP.pt')
            I1, I2 = GRID_IDX_DICT[well]['LGR1']['I1']-1-15, GRID_IDX_DICT[well]['LGR1']['I2']+15
            J1, J2 = GRID_IDX_DICT[well]['LGR1']['J1']-1-15, GRID_IDX_DICT[well]['LGR1']['J2']+15
            coarse = np.repeat(PRED['GLOBAL'][0,...][:,I1:I2,J1:J2,:,:],5,axis=-2).permute(-1,1,2,3,0)[...,None]
            x_LGR1 = torch.cat((data_LGR1['input'][...,:-1],coarse),axis=-1)
            x_LGR1 = x_LGR1.permute(0,4,1,2,3,5).to(device)
            x_LGR1[...,-1:] = NORMALIZER_DICT['input']['LGR1'].encode(x_LGR1.to(device)[...,-1:])
            pred = NORMALIZER_DICT['output']['LGR1'].decode(MODEL_DICT['LGR1'](x_LGR1)).cpu()
            lgr_dict['LGR1'] = pred
            y = data_LGR1['output'][...,:1].permute(0,4,1,2,3,5)
            true_dict['LGR1'] = y

            data_LGR2 = torch.load(f'dataset/dP_LGR2/{slope}_{idx}_LGR2_{well}_DP.pt')
            coarse = np.repeat(lgr_dict['LGR1'][0,...],2,axis=-2).permute(-1,1,2,3,0)[...,None]
            x_LGR2 = torch.cat((data_LGR2['input'][...,:-1],coarse),axis=-1)
            x_LGR2 = x_LGR2.permute(0,4,1,2,3,5).to(device)
            x_LGR2[...,-1:] = NORMALIZER_DICT['input']['LGR2'].encode(x_LGR2.to(device)[...,-1:])
            pred = NORMALIZER_DICT['output']['LGR2'].decode(MODEL_DICT['LGR2'](x_LGR2)).cpu()
            lgr_dict['LGR2'] = pred
            y = data_LGR2['output'][...,:1].permute(0,4,1,2,3,5)
            true_dict['LGR2'] = y

            data_LGR3 = torch.load(f'dataset/dP_LGR3/{slope}_{idx}_LGR3_{well}_DP.pt')
            coarse = lgr_dict['LGR2'][0,...].permute(-1,1,2,3,0)[...,None]
            x_LGR3 = torch.cat((data_LGR3['input'][...,:-1],coarse),axis=-1)
            x_LGR3 = x_LGR3.permute(0,4,1,2,3,5).to(device)
            x_LGR3[...,-1:] = NORMALIZER_DICT['input']['LGR3'].encode(x_LGR3.to(device)[...,-1:])
            pred = NORMALIZER_DICT['output']['LGR3'].decode(MODEL_DICT['LGR3'](x_LGR3)).cpu()
            lgr_dict['LGR3'] = pred
            y = data_LGR3['output'][...,:1].permute(0,4,1,2,3,5)
            true_dict['LGR3'] = y

            data_LGR4 = torch.load(f'dataset/dP_LGR4/{slope}_{idx}_LGR4_{well}_DP.pt')
            coarse = lgr_dict['LGR3'][0,...].permute(-1,1,2,3,0)[...,None]
            x = data_LGR4['input']
            x_LGR4 = torch.cat((x[...,:-1],coarse),axis=-1)
            x_LGR4 = x_LGR4.permute(0,4,1,2,3,5).to(device)
            x_LGR4[...,-1:] = NORMALIZER_DICT['input']['LGR4'].encode(x_LGR4.to(device)[...,-1:])
            pred = NORMALIZER_DICT['output']['LGR4'].decode(MODEL_DICT['LGR4'](x_LGR4)).cpu()
            lgr_dict['LGR4'] = pred
            y = data_LGR4['output'][...,:1].permute(0,4,1,2,3,5)
            true_dict['LGR4'] = y

            err = pred - y
            ERR_LIST.append(err.numpy())

# Save error in matrix

In [32]:
err = np.array(ERR_LIST).reshape(-1, 24, 40, 40, 50, 1)
print(err.shape)

(43, 24, 40, 40, 50, 1)


# Reload from trained weights

In [24]:
from lploss import *
LPloss = LpLoss(size_average=True)

from FNO4D import *
device = torch.device('cuda')
width = 28
model = torch.load( f"pre_trained_models/FNO4D-LGR4-DP.pt")
model.to(device)

FNO4d(
  (fc0): Linear(in_features=9, out_features=28, bias=True)
  (conv): Block4d(
    (conv0): SpectralConv4d()
    (conv1): SpectralConv4d()
    (conv2): SpectralConv4d()
    (conv3): SpectralConv4d()
    (w0): Conv1d(28, 28, kernel_size=(1,), stride=(1,))
    (w1): Conv1d(28, 28, kernel_size=(1,), stride=(1,))
    (w2): Conv1d(28, 28, kernel_size=(1,), stride=(1,))
    (w3): Conv1d(28, 28, kernel_size=(1,), stride=(1,))
    (fc1): Linear(in_features=28, out_features=112, bias=True)
    (fc2): Linear(in_features=112, out_features=1, bias=True)
  )
)

In [25]:
from datetime import datetime
from datetime import date

now = datetime.now()
today = date.today()
key = 'LGR4' 
day = today.strftime("%m%d")
current_time = now.strftime("%H%M")
specs = f'FNO4D-{key}-DP'
model_str = f'{day}-{current_time}-finetune{n_train}'
print(f'{specs}-{model_str}')

FNO4D-LGR4-DP-1107-1730-finetune16


In [26]:
with open(f"normalizer/input_normalizer_{key}_DP_val.pickle", 'rb') as f:
    input_normalizer = pickle.load(f)
    input_normalizer.cuda()
    
with open(f"normalizer/output_normalizer_{key}_DP_val.pickle", 'rb') as f:
    output_normalizer = pickle.load(f)
    output_normalizer.cuda()

In [27]:
from Adam import Adam
scheduler_step = 2
scheduler_gamma = 0.85
learning_rate = 1e-3

optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                        step_size=scheduler_step, 
                                        gamma=scheduler_gamma)

In [28]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(f'logs/')

# Finetune

In [ ]:
for ep in range(51,60):
    model.train()
    train_lp = 0, 0
    c = 0
    
    for data in train_loader:
        x, y, path = data['x'], data['y'], data['path']
        rand_idx = np.random.randint(err.shape[0]) # change this to the size of your saved error
        x[...,-1:] += torch.from_numpy(err[rand_idx,...])
        x, y = x[None,...].to(device), y[None,...][...,:1].to(device)
        
        optimizer.zero_grad()
        x[...,-1:] = input_normalizer_GLOBAL.encode(x[...,-1:])
        pred = model_global(x)
        pred = output_normalizer_GLOBAL.decode(pred)
        
        loss = LPloss(pred.reshape(1, -1), y.reshape(1, -1))
        train_lp += loss.item()
        
        loss.backward()
        optimizer.step()
        c += 1
        
        if c%100 ==0:
            writer.add_scalars('dp LPloss', {f'{model_str}_{specs}_train': loss.item()}, ep*n_train+c)
            print(f'ep: {ep}, iter: {c}, train lp: {loss.item():.4f}')

    scheduler.step()
    print('----------------------------------------------------------------------')

    torch.save(model_global, f'saved_models/{model_str}-{specs}-ep{ep}.pt')